# Part 3 (a) (Continued..)

Plan of action:

- Preprocess the data, using `load_and_preprocess`, and split the data into `train_texts`, `val_texts` and `val_text_70` $\equiv$ 900, 100 and 100 (systems). 
- The two validation sets `val_texts` and `val_text_70` have the same `shape` but:
    - In `val_texts` each system has the full 100 pairs of prey and predators
    - In `val_texts_70` each system has only the first 70 pairs of prey and predators
- We train the model on tokenised `train_texts`
- We validated the model by predicting the remaining 30 pair points in each of the 100 system in tokenised `val_texts_70`. 
- We then compare the predicted results from `val_texts_70` to the gruond truth data `val_texts` (or `true_val_values` obtained with `data_scale_split`)
- Just like for the untrained models we then want to compute MSE and RMSE 
- And report the loss/perplexity of each trained models

We are recomended to train our model up to 10,000 steps, but we have a budgeted number of flops overall for training $10^{17}$ and due to computational power required, we are going to proceed with fewer steps first, also to familiarise with the traing procedure, before increasing the number of steps and using HPC.

ESSENTIALLY

“We trained on 900 systems, validated on 100 full sequences for loss monitoring, and evaluated forecasting performance by generating future predictions given the first 70 steps from each validation sequence.”

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from src.preprocessor import*
from src.plotting import*
from sklearn.metrics import mean_absolute_error
from src.minimal_lora_eval import process_sequences
from src.qwen import tokenize_time_series_np
from src.minimal_lora_eval import evaluate_loss, process_sequences

In [ ]:
import torch
# Set seed
torch.manual_seed(23)

In [ ]:
from src.minimal_lora_eval import model, tokenizer # Import model and tokenizer from qwen with lora injected

In [ ]:
train_texts, val_texts, val_texts_70 = load_and_preprocess("data/lotka_volterra_data.h5") # preprocess the data

# split the data not converting to string, this will be used to compute the error in the validation set
_, true_val_values, _ = data_scale_split("data/lotka_volterra_data.h5") 

Tokenise with overlapping chucks

In [ ]:
""" 
Here we are:
1. Tokenising each time series using Qwen tokenizer
2. Chunking the tokenised output into sliding windows of tokens (for training)
3. Pad shorter sequences to a fixed length
"""

train_input_ids = process_sequences(train_texts, tokenizer, max_length=512, stride=256)
val_input_ids = process_sequences(val_texts, tokenizer, max_length=512, stride=512)

The reason why we stride our training dataset, is to generate more overlap during training, wich leads to more signal from each long sequence

Train the Model

In [ ]:
# TensorDataset allows you to wrap tensors into a dataset object
# DataLoader provides minibatch iteration over the dataset
from torch.utils.data import TensorDataset, DataLoader

# Accelerator from Hugging Face simplifies GPU/multi-device/mixed-precision training
from accelerate import Accelerator

# tqdm is used for progress bars — makes training loops easier to monitor
from tqdm import tqdm


In [ ]:
# Set hyperparameters
batch_size = 4  # Number of sequences per training batch
learning_rate = 1e-5 # Learning rate for the optimizer

# Set up the optimizer to update only trainable parameters (e.g., LoRA adapters + LM head bias)
optimizer = torch.optim.Adam(
    (p for p in model.parameters() if p.requires_grad), lr=learning_rate
)
# Wrap the training data into a TensorDataset and DataLoader for batching
train_dataset = TensorDataset(train_input_ids)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize Hugging Face Accelerator for device/mixed-precision handling
accelerator = Accelerator()

# Prepare model, optimizer, and dataloader with Accelerator
model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)

In [ ]:


# Set model to training mode
model.train()
steps = 0

max_steps = 10000
# Start training loop
while steps < max_steps:  # QPLPPP = 10,000 steps
    for (batch,) in tqdm(train_loader, desc=f"Steps {steps}"):

         # Reset gradients
        optimizer.zero_grad()

        # Forward pass with labels = input_ids for language modeling (next-token prediction)
        outputs = model(batch, labels=batch)
        loss = outputs.loss

        # Backpropagation using Accelerator
        accelerator.backward(loss)
        optimizer.step()

        # Step counter
        steps += 1

        # Break loop after desired number of steps
        if steps >= max_steps:
            break

Computing validation loss before prediction

Collecting loss and perplexity

In [ ]:
print(f"After training with {max_steps} steps")
print(f"Training loss: {loss.item():.4f}")
perplexity = torch.exp(loss).item()
print(f"Perplexity: {perplexity:.2f}")


Checking if weights were updated during training

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name}: {param.shape}")


Saving LoRA Weights only (Efficitent checkpoints).

Save the LoRA adapter weights (not the full Qwen model).

In [ ]:
# Extract LoRA-only weights
lora_state_dict = {
    name: param.cpu()
    for name, param in model.named_parameters()
    if param.requires_grad
}

torch.save(lora_state_dict, "trained_lora_3a_10000/lora_weight_matrices_10000.pt")


In [ ]:
val_dataset = TensorDataset(val_input_ids)
val_loader = DataLoader(val_dataset, batch_size=4)
val_loader = accelerator.prepare(val_loader)

val_loss = evaluate_loss(model, val_loader)
perplexity_val = np.exp(val_loss)
print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Perplexity: {perplexity_val}")


Saving loss and perplexity experienced in training and validation

In [ ]:
loss_perplexity_df = pd.DataFrame({
    "Loss": [loss.item(), val_loss],  
    "Perplexity": [perplexity, perplexity_val],  

}, ['Training', 'Validation'])
loss_perplexity_df.to_csv("trained_lora_3a_10000/Loss_Perplexity_Lora_10000", index = True)

In [ ]:
def sequence_length_array(tokenized_string):

    
    max_lengths = np.array([entry["input_ids"].shape[1] for entry in tokenized_string])
    return max_lengths

In [ ]:
# Tokeniseing data series
tok_val_texts = tokenize_time_series_np(val_texts)
tok_val_texts_70 = tokenize_time_series_np(val_texts_70)


Now that the model is trained and validated we can perform prdictions using `model.generate()`

In [ ]:
# Unwrap the model from Accelerate (optional) and move to GPU
model = accelerator.unwrap_model(model)
model.to("cuda")
model.eval()

predictions_encoded = []

# Loop over each validation sample for generation
for i in tqdm(range(len(tok_val_texts_70)), desc="Generating predictions"):
    
    # Extract tokenized input and attention mask for system i
    input_ids = tok_val_texts_70[i]["input_ids"].to(model.device)
    attention_mask = tok_val_texts_70[i]["attention_mask"].to(model.device)

    # Use model.generate() to predict the next tokens
    output = model.generate(
        input_ids=input_ids,                 # Input sequence (first 70 timesteps)
        attention_mask=attention_mask,       # Mask to ignore padding tokens
        min_new_tokens=350,                   # Force at least 30 new tokens to be generated
        max_new_tokens=400,                   # Upper bound to avoid runaway generation
        do_sample=False,                     # Use deterministic decoding (greedy search)
        eos_token_id=tokenizer.eos_token_id  # Stop generation at EOS if it appears
    )

    # Append generated output to the list (will decode later)
    predictions_encoded.append(output)


In [ ]:
predictions_decoded = []

for i in range(len(tok_val_texts_70)):
    decoded_output = string_to_array(tokenizer.decode(predictions_encoded[i][0], skip_special_tokens=True))
    predictions_decoded.append(decoded_output)

predicted_output = []
true_values = []
# Collecting the last 30 pair from each array
for i in range(len(predictions_decoded)):
    sub_output = predictions_decoded[i]
    sub_output_min_30 = sub_output[70:100]
    true_v = true_val_values[i]
    true_v_min_30 = true_v[70:]
    predicted_output.append(sub_output_min_30)
    true_values.append(true_v_min_30)

# Computing MSE
MSE_values = []

for i in range(len(true_values)):

    # Truncate to match the shorter list
    min_length = min(len(predicted_output[i]), len(true_values[i]))
    pr_out = predicted_output[i][:min_length]
    true_val = true_values[i][:min_length]

    # Compute the MSE
    mse_prey = mean_absolute_error(pr_out[:,0], true_val[:,0]) # Computing MSE for prey
    mse_predator = mean_absolute_error(pr_out[:,1], true_val[:,1]) # Computing MSE for predator
    MSE_values.append([mse_prey, mse_predator])

# Computing RMSE
RMSE_values = np.sqrt(MSE_values)

df_MSE_values = pd.DataFrame({
    "system_id": np.arange(len(MSE_values)),  
    "MSE for prey": np.array(MSE_values)[:, 0],  # Flatten prey values
    "MSE for predator": np.array(MSE_values)[:, 1]  # Flatten predator values
})
df_RMSE_values = pd.DataFrame({
    "system_id": np.arange(len(MSE_values)),  
    "RMSE for prey": RMSE_values[:, 0],  # Flatten prey values
    "RMSE for predator": RMSE_values[:, 1]  # Flatten predator values
})

# Computing the error for each system

error_per_system = []

for i in range(len(true_values)):

    # Truncate to match the shorter list
    min_length = min(len(predicted_output[i]), len(true_values[i]))
    pr_out = predicted_output[i][:min_length]
    true_val = true_values[i][:min_length]

    # Computing errors
    prey_error = true_val[:,0] - pr_out[:,0]
    predator_error = true_val[:,1] - pr_out[:,1]
    
    # Collecting errors into a list
    error_per_system.append([prey_error, predator_error])

print('Length of error_per_system:',len(error_per_system))

from src.plotting import PlotProject
PlotProject.plot_hist_MSE(df_MSE_values,bins=30) # Plotting MSE distribution
PlotProject.plot_hist_RMSE(df_RMSE_values,bins=30) # Plotting RMSE distribution
PlotProject.plot_error_hist_system(error_per_system, 0, bins = 30)
pred_vs_true_visualisation(predictions_decoded, true_val_values, 0)

Save results

In [ ]:
np.savez("trained_lora_3a_10000/predictions_decoded_trained_lora_3a_10000.npz", *predictions_decoded)
MSE_loaded = np.save("trained_lora_3a_10000/MSE_values_3a_10000.npy", np.array(MSE_values))
np.save('trained_lora_3a_10000/RMSE_values_3a_10000', RMSE_values)
np.savez("trained_lora_part_3a/error_per_system_5000.npz", *error_per_system)

Reload LoRA weight matrices

In [ ]:
#weight_matrices_10000 = model.load_state_dict(torch.load("trained_lora_3a_10000/lora_weight_matrices_10000.pt"), strict=False)


In [ ]:
for i in range(15,20): 
    pred_vs_true_visualisation(predictions_decoded, true_val_values, i)